In [ ]:
# import all dependencies
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
# executable_path = {"executable_path": "chromedriver.exe"}
# browser = Browser("chrome", **executable_path, headless=False)
from selenium import webdriver #for mac
# browser = Browser("chrome", headless=False)#for mac

In [ ]:
browser = Browser("chrome", headless=False)#for mac

In [ ]:
# Scrape the NASA Mars News site
nasa_mars_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_mars_url)
time.sleep(1)
nasa_mars_html = browser.html
nasa_mars_soup = BeautifulSoup(nasa_mars_html, 'html.parser')

news_list = nasa_mars_soup.find('ul', class_='item_list')
first_item = news_list.find('li', class_='slide')
nasa_mars_headline = first_item.find('div', class_='content_title').text
nasa_mars_para = first_item.find('div', class_='article_teaser_body').text
print(nasa_mars_headline)
print(nasa_mars_para)

In [ ]:
# visit the JPL website and scrape the featured image
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featured_image_url)
time.sleep(1)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)
expand = browser.find_by_css('a.fancybox-expand')
expand.click()
time.sleep(1)

featured_image_html = browser.html
featured_image_soup = BeautifulSoup(featured_image_html, 'html.parser')

featured_image_relative = featured_image_soup.find('img', class_='fancybox-image')['src']
featured_image_path = f'https://www.jpl.nasa.gov{featured_image_relative}'
print(featured_image_path)

In [ ]:
# visit the mars weather report twitter and scrape the latest tweet
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_weather_url)
time.sleep(1)
mars_weather_html = browser.html
mars_weather_soup = BeautifulSoup(mars_weather_html, 'html.parser')

mars_weather_twitter = mars_weather_soup.find('ol', class_='stream-items')
mars_weather = mars_weather_twitter.find('p', class_="tweet-text").text
print(mars_weather)

In [ ]:
# visit space facts and scrap the mars facts table
mars_facts_url = 'https://space-facts.com/mars/'
browser.visit(mars_facts_url)
time.sleep(1)
mars_facts_html = browser.html
mars_facts_soup = BeautifulSoup(mars_facts_html, 'html.parser')
#mars_facts_soup.select_one("")

In [ ]:
mars_facts_table = mars_facts_soup.find('table', class_='tablepress tablepress-id-comp-mars blue-table')
print(mars_facts_table)
column1 = mars_facts_table.find_all('td', class_='column-1')
column2 = mars_facts_table.find_all('td', class_='column-2')

facets = []
values = []

for row in column1:
    facet = row.text.strip()
    facets.append(facet)
    
for row in column2:
    value = row.text.strip()
    values.append(value)
    
mars_facts = pd.DataFrame({
    "Facet":facets,
    "Value":values
    })

mars_facts_html = mars_facts.to_html(header=False, index=False)
mars_facts

In [ ]:
# scrape images of Mars' hemispheres from the USGS site
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
mars_hemisphere_dicts = []

for i in range(1,9,2):
    mars_hemisphere_dict = {}
    
    browser.visit(mars_hemisphere_url)
    time.sleep(1)
    mars_hemisphere_html = browser.html
    mars_hemisphere_soup = BeautifulSoup(mars_hemisphere_html, 'html.parser')
    mars_hemisphere_name_links = mars_hemisphere_soup.find_all('a', class_='product-item')
    mars_hemisphere_name = mars_hemisphere_name_links[i].text.strip('Enhanced')
    
    detail_links = browser.find_by_css('a.product-item')
    detail_links[i].click()
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(1)
    browser.windows.current = browser.windows[-1]
    mars_hemisphere_img_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()
    
    mars_hemisphere_img_soup = BeautifulSoup(mars_hemisphere_img_html, 'html.parser')
    mars_hemisphere_img_path = mars_hemisphere_img_soup.find('img')['src']

    print(mars_hemisphere_name)
    mars_hemisphere_dict['title'] = mars_hemisphere_name.strip()
    
    print(mars_hemisphere_img_path)
    mars_hemisphere_dict['img_url'] = mars_hemisphere_img_path

    mars_hemisphere_dicts.append(mars_hemisphere_dict)